In [33]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from tqdm import tqdm

class MaqamCNN1(nn.Module):
    def __init__(self):
        super(MaqamCNN1, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=30, out_channels=64, kernel_size=(3,3), padding="same")
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=(1,1))
        self.dropout1 = nn.Dropout(p=0.1)
        
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(1,1), padding="valid")
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=(1,1))
        self.dropout2 = nn.Dropout(p=0.2)

        self.fc1 = nn.Linear(30016, 512)
        self.dropout3 = nn.Dropout(p=0.2)

        self.fc2 = nn.Linear(512, 265)
        self.dropout4 = nn.Dropout(p=0.2)

        self.fc3 = nn.Linear(265, 100)
        self.dropout5 = nn.Dropout(p=0.2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        x = self.pool1(x)
        x = self.dropout1(x)
        
        x = F.relu(self.conv2(x))
        x = self.bn2(x)
        x = self.pool2(x)
        x = self.dropout2(x)
        
        x = x.view(x.size(0), -1)
        
        x = F.relu(self.fc1(x))
        x = self.dropout3(x)
        
        x = F.relu(self.fc2(x))
        x = self.dropout4(x)

        x = self.fc3(x)
        x = self.dropout5(x)
        x = F.softmax(x, dim=1)
        return x


In [34]:
class CNN_LSTM(nn.Module):
    def __init__(self):
        super(CNN_LSTM, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=30, out_channels=64, kernel_size=(3,3), padding="same")
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=(1,1))
        self.dropout1 = nn.Dropout(p=0.1)
        
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(1,1), padding="valid")
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=(1,1))
        self.dropout2 = nn.Dropout(p=0.2)

        self.lstm = nn.LSTM(input_size=64, hidden_size=128, num_layers=2, batch_first=True)
        self.dropout6 = nn.Dropout(p=0.25)

        self.fc1 = nn.Linear(128, 64)
        self.dropout3 = nn.Dropout(p=0.2)

        self.fc2 = nn.Linear(64, 32)
        self.dropout4 = nn.Dropout(p=0.2)

        self.fc3 = nn.Linear(32, 8)
        self.dropout5 = nn.Dropout(p=0.2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        x = self.pool1(x)
        x = self.dropout1(x)
        
        x = F.relu(self.conv2(x))
        x = self.bn2(x)
        x = self.pool2(x)
        x = self.dropout2(x)

        # Reshape the CNN output to match LSTM input
        batch_size, channels, height, width = x.size()
        x = x.view(batch_size, channels, height * width)
        x = x.permute(0, 2, 1)  # Permute to (batch_size, sequence_length, input_size)

        x, _ = self.lstm(x)

        x = self.fc1(x[:, -1, :])
        
        x = F.relu(self.fc2(x))
        x = self.dropout3(x)
        
        x = F.relu(self.fc3(x))
        # x = self.dropout4(x)

        # x = self.fc3(x)
        # x = self.dropout5(x)
        x = F.softmax(x, dim=1)
        return x


In [35]:
import torch.nn as nn
import torch.nn.functional as F

class MFCC_LSTM(nn.Module):
    def __init__(self):
        super(MFCC_LSTM, self).__init__()
        
        self.lstm = nn.LSTM(input_size=30, hidden_size=64, num_layers=1, batch_first=True)
        
        self.fc1 = nn.Linear(64, 64)
        self.dropout1 = nn.Dropout(p=0.2)
        
        self.fc2 = nn.Linear(64, 8)

    def forward(self, x):
        x = torch.transpose(x, 1, 2)
        # print(x.shape)
        _, (hn, cn) = self.lstm(x)

        x = hn[-1]  # Take the last hidden state of the LSTM
        
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x


In [36]:
import os
import torchaudio
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import librosa
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

class MaqamDataset(Dataset):
    def __init__(self, mode='train', transform=None, cache_file='1.pkl', test_size=0.2):
        self.mode = mode
        self.transform = transform
        if mode == 'train' or mode == 'val':
            self.data_dir = "/home/faisal/Desktop/MAQAMAT/Fullynewdataset"
        else:
            self.data_dir = "/home/faisal/Desktop/MAQAMAT/Test1"
        #['Ajam', 'Bayat', 'Hijaz', 'Kurd', 'Nahawand', 'Rast', 'Saba', 'Seka']
        self.maqams = ['Ajam', 'Bayat', 'Hijaz', 'Kurd', 'Nahawand', 'Rast', 'Saba', 'Seka']
        self.audio_list = self._load_audio_list()
        
        # Split the dataset into training and validation sets using train_test_split method
        if(self.mode == 'train'):
            train_list, val_list = train_test_split(self.audio_list, test_size=test_size, random_state=42, stratify=[label for (_, label) in self.audio_list])
            self.audio_list = train_list
        elif(self.mode == 'val'):
            train_list, val_list = train_test_split(self.audio_list, test_size=test_size, random_state=42, stratify=[label for (_, label) in self.audio_list])
            self.audio_list = val_list
        
        self.cache_file = cache_file
        self.data = self._load_data_from_cache_or_compute()
        # self.pad_to_max_length(1440000)

    def _load_audio_list(self):
        audio_list = []
        for i, maqam in enumerate(self.maqams):
            label_dir = os.path.join(self.data_dir, maqam)
            audio_list += [(os.path.join(label_dir, audio_name), i) for audio_name in os.listdir(label_dir) if audio_name.endswith('.wav')]
        return audio_list

    def __len__(self):
        return len(self.audio_list)

    def __getitem__(self, idx):
        audio_path, label_idx = self.audio_list[idx]
        waveform, sample_rate = torchaudio.load(audio_path)
        waveform = waveform[0] # only keep the first channel
        if self.transform:
            waveform = self.transform(waveform)
        mfcc = self.compute_mfcc(waveform).T
        mfcc = torch.from_numpy(mfcc).float()
        return mfcc, label_idx
    
    def pad_to_max_length(self, max_length):
        for i in range(len(self)):
            padded_data = F.pad(self.data[i][0], (0, max_length - len(self.data[i][0])), 'constant', 0)
            self.data[i] = (padded_data, self.data[i][1])

    def compute_mfcc(self, waveform):
        # Compute the MFCC of the waveform
        n_fft = 1024*8
        hop_length = 512*2
        n_mels = 100
        sr = 16000
        n_mfcc = 30
        waveform = waveform.numpy()  # Convert PyTorch tensor to NumPy array
        mfcc = librosa.feature.mfcc(y=waveform, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, n_mfcc=n_mfcc)
        mfcc = np.transpose(mfcc)
        mfcc = mfcc.astype(np.float32)  # Ensure data type is compatible with np.issubdtype()
        return mfcc
    
    def _load_data_from_cache_or_compute(self):
        if os.path.isfile(self.cache_file):
            print(f'Loading data from cache file: {self.cache_file}')
            with open(self.cache_file, 'rb') as f:
                return pickle.load(f)
        else:
            print(f'Cache file not found. Computing data from scratch and saving to cache file: {self.cache_file}')
            data = [self.__getitem__(i) for i in range(len(self))]
            with open(self.cache_file, 'wb') as f:
                pickle.dump(data, f)
            return data


In [37]:
import torch
from torch.utils.data import DataLoader
import librosa
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

max_length = 1440000

def MFCC_plot(mfcc):
        plt.figure(figsize=(10, 4))
        mfcc = mfcc.detach().numpy()
        mfcc = mfcc.mean(axis=2).T
        librosa.display.specshow(mfcc, x_axis='time')
        plt.colorbar()
        plt.title('MFCC')
        plt.tight_layout()
        plt.show()

def custom_collate(batch):
    inputs, labels = zip(*batch)
    max_frames = max([m.shape[1] for m in inputs])
    padded_mfcc = []
    for m in inputs:
        pad_width = ((0, 0), (0, max_frames - m.shape[1]))
        padded_m = np.pad(m, pad_width=pad_width, mode='constant')
        padded_mfcc.append(padded_m)

    padded_mfcc = torch.from_numpy(np.array(padded_mfcc)).float()
    labels = torch.tensor(labels)
    return padded_mfcc, labels



In [38]:
# Define training and validation datasets with specified test size
train_dataset = MaqamDataset(mode='train', test_size=0.2)
val_dataset = MaqamDataset(mode='val', test_size=0.2)

# Define training and validation data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate)

Loading data from cache file: 1.pkl
Loading data from cache file: 1.pkl


In [39]:
torch.cuda.init()
torch.cuda.empty_cache()
option = 2

In [40]:
if(option == 1):
    torch.cuda.init()
    torch.cuda.empty_cache()
    # Initialize model and define loss function and optimizer
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = MFCC_LSTM().to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # Train the model for a specified number of epochs
    num_epochs = 3
    print("Starting training")
    for epoch in range(num_epochs):
        # Set the model to training mode for the current epoch
        model.train()

        # Create a progress bar for the train_loader loop
        train_loader_tqdm = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)

        # Initialize variables to track the loss and number of correct predictions
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        for i, data in enumerate(train_loader_tqdm):
            inputs, targets = data  # MFCCs and labels
            targets = targets.to(device)
            inputs = inputs.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            # Update the loss and accuracy metrics
            running_loss += loss.item()
            _, predicted_labels = torch.max(outputs, 1)
            correct_predictions += (predicted_labels == targets).sum().item()
            total_samples += len(targets)

            # Update the progress bar description
            train_loader_tqdm.set_postfix({'Loss': running_loss / (i + 1), 'Accuracy': 100 * correct_predictions / total_samples})

        # Calculate and print average loss and accuracy for the current epoch
        avg_loss = running_loss / len(train_loader)
        avg_accuracy = 100 * correct_predictions / total_samples
        print(f'Epoch {epoch + 1}/{num_epochs}: Train Loss={avg_loss:.5f}, Train Accuracy={avg_accuracy:.5f}%')

        # Validation loop
        model.eval()
        with torch.no_grad():
            val_loss = 0.0
            total_correct = 0
            total_samples = 0

            # Create a progress bar for the val_loader loop
            val_loader_tqdm = tqdm(val_loader, desc='Validation', leave=False)

            for data in val_loader_tqdm:
                inputs, targets = data  # MFCCs and labels
                targets = targets.to(device)
                inputs = inputs.cuda()
                outputs = model(inputs)
                val_loss += criterion(outputs, targets).item() * len(targets)

                _, predicted_labels = torch.max(outputs, 1)
                total_correct += (predicted_labels == targets).sum().item()
                total_samples += len(targets)

                # Update the progress bar description
                val_loader_tqdm.set_postfix({'Validation Loss': val_loss / total_samples, 'Validation Accuracy': 100 * total_correct / total_samples})

            val_loss /= len(val_dataset)
            val_acc = float(total_correct) / total_samples

        print(f'Epoch {epoch + 1}/{num_epochs} validation: val_loss={val_loss:.5f}, val_acc={100*val_acc:.5f}%')

    # Save the trained model
    torch.save(model.state_dict(), 'readerstest.pth')

Starting training


Epoch 1/3: Train Loss=2.07732, Train Accuracy=15.88694%


Epoch 1/3 validation: val_loss=2.07291, val_acc=24.51362%


Epoch 2/3: Train Loss=2.06725, Train Accuracy=24.85380%


Epoch 2/3 validation: val_loss=2.06316, val_acc=26.07004%


Epoch 3/3: Train Loss=2.04549, Train Accuracy=26.02339%


Epoch 3/3 validation: val_loss=2.03862, val_acc=25.68093%


In [41]:
if(option == 1):
    # Test the model on new data
    test_dataset = MaqamDataset(mode='test', cache_file='1.pkl')
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate)

    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for data in test_loader:
            inputs, targets = data
            targets = targets.to(device)
            # print("targets: ",targets)
            inputs = inputs.cuda()
            outputs = model(inputs)
            # Convert the list to a tensor (if it's not already a tensor)
            probs_tensor = torch.tensor(outputs[1])

            # Find the index of the class with the highest probability
            predicted_label_index = torch.argmax(probs_tensor)
            # print("outputs: ",predicted_label_index)
            _, predicted_labels = torch.max(outputs, 1)
            # print("predicted_labels: ",outputs)
            total_correct += (predicted_labels == targets).sum().item()
            total_samples += len(targets)

        test_acc = 100 * float(total_correct) / total_samples

    print(f'Test accuracy: {test_acc:.5f}%')

Loading data from cache file: 1.pkl


/tmp/ipykernel_24980/3981001201.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  probs_tensor = torch.tensor(outputs[1])


predicted_labels:  tensor([[0.2298, 0.0701, 0.2310, 0.1162, 0.1376, 0.0751, 0.0602, 0.0800],
        [0.1671, 0.0871, 0.2256, 0.1062, 0.1463, 0.1190, 0.0577, 0.0909],
        [0.1489, 0.1055, 0.1119, 0.0722, 0.1947, 0.1663, 0.1238, 0.0766],
        [0.1596, 0.0803, 0.2251, 0.0985, 0.1288, 0.0986, 0.0724, 0.1366],
        [0.1604, 0.0678, 0.1455, 0.0828, 0.1432, 0.1724, 0.0783, 0.1496],
        [0.1798, 0.0932, 0.2903, 0.1504, 0.0754, 0.0601, 0.0465, 0.1043],
        [0.1824, 0.0720, 0.3236, 0.1259, 0.0886, 0.0650, 0.0573, 0.0851],
        [0.1514, 0.0878, 0.3118, 0.1368, 0.0976, 0.0780, 0.0634, 0.0734],
        [0.2378, 0.0910, 0.1193, 0.0611, 0.1376, 0.1977, 0.0798, 0.0758],
        [0.1969, 0.0546, 0.1749, 0.0919, 0.1412, 0.0806, 0.1152, 0.1447],
        [0.1485, 0.1038, 0.1681, 0.0930, 0.1204, 0.1962, 0.0550, 0.1150],
        [0.1769, 0.0920, 0.2795, 0.1567, 0.0808, 0.0809, 0.0379, 0.0953],
        [0.1542, 0.0611, 0.2042, 0.0686, 0.1589, 0.0604, 0.2260, 0.0665],
        [0.1566, 0.

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
if(option == 2):
    # Rest of the model definition and data loaders

    # Initialize model and define loss function and optimizer
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = MFCC_LSTM().to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Train the model for a specified number of epochs
    num_epochs = 40
    patience = 5  # Number of epochs to wait for improvement before early stopping
    best_val_loss = float('inf')
    best_model_state_dict = None
    no_improvement_epochs = 0

    print("Starting training")
    for epoch in range(num_epochs):
        # Set the model to training mode for the current epoch
        model.train()

        # Training loop
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        for i, data in enumerate(tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)):
            inputs, targets = data  # MFCCs and labels
            targets = targets.to(device)
            inputs = inputs.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            # Update the loss and accuracy metrics
            running_loss += loss.item()
            _, predicted_labels = torch.max(outputs, 1)
            correct_predictions += (predicted_labels == targets).sum().item()
            total_samples += len(targets)

        # Calculate and print average loss and accuracy for the current epoch
        avg_loss = running_loss / len(train_loader)
        avg_accuracy = 100 * correct_predictions / total_samples
        print(f'Epoch {epoch + 1}/{num_epochs}: Train Loss={avg_loss:.5f}, Train Accuracy={avg_accuracy:.5f}%')

        # Validation loop
        model.eval()
        val_loss = 0.0
        total_correct = 0
        total_samples = 0

        with torch.no_grad():
            for data in tqdm(val_loader, desc='Validation', leave=False):
                inputs, targets = data  # MFCCs and labels
                targets = targets.to(device)
                inputs = inputs.to(device)
                outputs = model(inputs)
                val_loss += criterion(outputs, targets).item() * len(targets)

                _, predicted_labels = torch.max(outputs, 1)
                total_correct += (predicted_labels == targets).sum().item()
                total_samples += len(targets)

        val_loss /= len(val_dataset)
        val_acc = 100 * total_correct / total_samples

        print(f'Epoch {epoch + 1}/{num_epochs} validation: val_loss={val_loss:.5f}, val_acc={val_acc:.5f}%')

        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state_dict = model.state_dict()
            no_improvement_epochs = 0
        else:
            no_improvement_epochs += 1

        if no_improvement_epochs >= patience:
            print("Early stopping. No improvement in validation loss for {} epochs.".format(patience))
            break

    # Load the best model state dict
    if best_model_state_dict is not None:
        model.load_state_dict(best_model_state_dict)

    # Test the model on the test dataset
    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for data in tqdm(test_loader, desc='Testing', leave=False):
            inputs, targets = data
            targets = targets.to(device)
            inputs = inputs.to(device)
            outputs = model(inputs)

            _, predicted_labels = torch.max(outputs, 1)
            total_correct += (predicted_labels == targets).sum().item()
            total_samples += len(targets)

    test_acc = 100 * total_correct / total_samples
    print(f'Test Accuracy: {test_acc:.5f}%')

    # Save the trained model
    torch.save(model.state_dict(), 'readerstest.pth')


Starting training


Epoch 1/40: Train Loss=2.07646, Train Accuracy=15.49708%


Epoch 1/40 validation: val_loss=2.07148, val_acc=24.90272%


Epoch 2/40: Train Loss=2.06403, Train Accuracy=19.39571%


Epoch 2/40 validation: val_loss=2.05810, val_acc=22.17899%


...

Epoch 23/40: Train Loss=1.59850, Train Accuracy=70.17544%


Epoch 23/40 validation: val_loss=1.89804, val_acc=35.40856%


Epoch 24/40: Train Loss=1.56488, Train Accuracy=75.04873%


Epoch 24/40 validation: val_loss=1.90165, val_acc=35.79767%
Early stopping. No improvement in validation loss for 5 epochs.


Test Accuracy: 68.75000%
